Dynamic Programming

In [1]:
import os
import numpy as np
import cv2
import math

In [2]:
left_img = cv2.imread('view1.png', 0)  #read it as a grayscale image
right_img = cv2.imread('view5.png', 0)

In [3]:
#Disparity Computation for Left Image

In [4]:
OcclusionCost = int(20)

#For Dynamic Programming you have build a cost matrix. Its dimension will be numcols x numcols

In [5]:
CostMatrix = np.zeros((left_img.shape[1],left_img.shape[1]),dtype=np.int_)

#create a direction matrix... This is important in Dynamic Programming. You need to know which direction you need traverse

In [6]:
DirectionMatrix = np.zeros((left_img.shape[1],left_img.shape[1]),dtype=np.int_)

In [7]:
(numrows,numcols) = left_img.shape
DisparityMatrix = np.zeros(left_img.shape)

In [8]:
numcols= int(numcols)
numrows=int(numrows)

In [9]:
#creating and initialising the CostMatrix
for i in range(0,numcols):
    CostMatrix[i,0]=OcclusionCost*i
    CostMatrix[0,i]=i*OcclusionCost

#Using the pseudocode from "A Maximum likelihood Stereo Algorithm" paper given as reference
#computing the costMatrix and directionMatrix

In [10]:
CostMatrixInit = np.copy(CostMatrix)

In [11]:
def subSeq(M):
    p=int(M.shape[1]-1)
    q=int(M.shape[1]-1)
    matchSeq = np.zeros((1,numcols),dtype=np.int_)
    while((p>0) and (q>0)):
        if(M[p,q]==1):
            popValue = q-p
            #print("numcols/q: ",numcols,q)
            #matchSeq[0,min((numcols-q),(numcols-1))]=popValue
            matchSeq[0,q]=popValue
            p-=1
            q-=1
        if(M[p,q]==2):
            p-=1
        if(M[p,q]==3):
            """ if(q==(numcols-1)):
                popValue=0
            else:
                popValue=matchSeq[0,q+1]
            q-=1
            #print("numcols/q: ",numcols,q)
            #matchSeq[0,min((numcols-q),(numcols-1))]=popValue
            """
            q-=1
            popValue=0
            matchSeq[0,q]=popValue
        #matchSeqRevOrder.append(popValue)
    return matchSeq

In [12]:
#k --> for processing one row of the image at a time
def getDPDisparityMatrix(numrows,numcols,CostMatrixInit,img1,img2):
    DisparityMatrix = np.zeros((numrows,numcols),dtype=np.int_)
    for k in range(0,numrows):
        imgRow=k
        CostMatrix = np.copy(CostMatrixInit)
        #i,j --> n --> no of cols of the image
        for i in range(1,numcols):
            for j in range(1,numcols):
                intensityDiff = int(img1[imgRow,max(i-1,0)]) - int(img2[imgRow,max(j-1,0)])
                min1 = CostMatrix[max(i-1,0),max(j-1,0)] + int(math.fabs(intensityDiff))
                min2 = CostMatrix[max(i-1,0),j] + OcclusionCost
                min3 = CostMatrix[i,max(j-1,0)] + OcclusionCost
                cmin=min(min1,min2,min3)
                CostMatrix[i,j]= cmin
                if min1==cmin:
                    DirectionMatrix[i,j]=1
                elif min2==cmin:
                    DirectionMatrix[i,j]=2
                else:
                    DirectionMatrix[i,j]=3
        #at the end of each iteration, process the Direction Matrix
        m = subSeq(DirectionMatrix)
        np.copyto(DisparityMatrix[k,:],m)
    return DisparityMatrix

In [13]:
from matplotlib import pyplot as plt
def plot_me(img):
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.show()

In [14]:
DM1 = getDPDisparityMatrix(numrows,numcols,CostMatrixInit,img1=left_img,img2=right_img)

In [19]:
plot_me(DM1)

In [20]:
DM2 = getDPDisparityMatrix(numrows,numcols,CostMatrixInit,img2=left_img,img1=right_img)

In [21]:
plot_me(DM2)

In [23]:
cv2.imwrite('DisparityMap using DP -1.png',DM1)
cv2.imwrite('DisparityMap using DP -2.png',DM2)

True